In [99]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def time_transfermarkt(url, nome_time):
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36',
                                  'Referer': 'https://www.transfermarkt.com.br/'})
    content = r.content
    
    soup = BeautifulSoup(content)
    
    dados = []
    
    # itera sobre todas as linhas da tabela do time
    for linha in soup.find_all('tr', class_=['even', 'odd']):
        preco_str = linha.find_all('td')[-1].text
        
        # regex para extrair o valor do jogador
        busca = re.search('\d+\,?\d+', preco_str)
        if busca == None:
            continue
        
        preco = float(busca.group().replace(',','.'))
        
        # transforma preços em milhares para milhões
        if 'mil' in preco_str:
            preco /= 1000
        
        col_nome = linha.find_all('a')
        nome = ''
        if len(col_nome) > 2:
            nome = col_nome[2]
        else:
            nome = col_nome[0]

        link = nome['href']
        nome = nome.text.strip()

        posicao = linha.find_all('td')[4].text.strip()

        idade = linha.find_all('td')[-4].text
        
        dados.append({'Jogador':nome, 'Posição':posicao, 'Preço':preco, 'Time': nome_time, 'Idade': idade, 'Link': link})
    
    return dados

In [96]:
def cria_tabela_transfermarkt(times):
    from time import sleep
    dados = []
    for url, time in times:
        dados += time_transfermarkt(url, time)
        sleep(1)
    
    return pd.DataFrame(dados)

In [11]:
import re
import pandas as pd

def cria_tabela_sofascore(string):
    regex_times = re.compile('[A-zÀ-ú ]+\n')
    regex_nomes = re.compile('[A-zÀ-ú ]+\t')
    regex_ratings = re.compile('\d.\d\d')
    regex_colocacao = re.compile('\d+\t')

    times = re.findall(regex_times, string)
    nomes = re.findall(regex_nomes, string)
    ratings = re.findall(regex_ratings, string)
    colocacao = re.findall(regex_colocacao, string)
    
    dados = []
    for time, nome, rating, colocacao in zip(times, nomes, ratings, colocacao):
        dados.append({'Time': time[:-1], 'Jogador': nome[:-1], 'Rating': rating, 'Colocação': colocacao[:-1]})
        
    return pd.DataFrame(dados)

In [295]:
temporada = 2020 # no transfermarkt, 2020 se refere a temporada 2021

america_mg = (f'https://www.transfermarkt.com.br/america-mineiro/kader/verein/2863/plus/0/galerie/0?saison_id={temporada}', 'América Mineiro')
athletico = (f'https://www.transfermarkt.com.br/athletico-paranaense/kader/verein/679/plus/0/galerie/0?saison_id={temporada}', 'Athletico')
atletico_go = (f'https://www.transfermarkt.com.br/atletico-goianiense/kader/verein/15172/plus/0/galerie/0?saison_id={temporada}', 'Atlético Goianiense')
atletico_mg = (f'https://www.transfermarkt.com.br/atletico-mineiro/kader/verein/330/plus/0/galerie/0?saison_id={temporada}', 'Atlético Mineiro')
ceara = (f'https://www.transfermarkt.com.br/ceara-sc/kader/verein/2029/plus/0/galerie/0?saison_id={temporada}', 'Ceará')
corinthians = (f'https://www.transfermarkt.com.br/sc-corinthians/kader/verein/199/plus/0/galerie/0?saison_id={temporada}', 'Corinthians')
cuiaba = (f'https://www.transfermarkt.com.br/cuiaba-ec/kader/verein/28022/plus/0/galerie/0?saison_id={temporada}', 'Cuiabá')
fluminense = (f'https://www.transfermarkt.com.br/fluminense-fc/kader/verein/2462/plus/0/galerie/0?saison_id={temporada}', 'Fluminense')
flamengo = (f'https://www.transfermarkt.com.br/cr-flamengo/kader/verein/614/saison_id/{temporada}', 'Flamengo')
fortaleza = (f'https://www.transfermarkt.com.br/fortaleza-ec/kader/verein/10870/plus/0/galerie/0?saison_id={temporada}', 'Fortaleza')
internacional = (f'https://www.transfermarkt.com.br/sc-internacional/kader/verein/6600/plus/0/galerie/0?saison_id={temporada}', 'Internacional')
juventude = (f'https://www.transfermarkt.com.br/ec-juventude/kader/verein/10492/plus/0/galerie/0?saison_id={temporada}', 'Juventude')
palmeiras = (f'https://www.transfermarkt.com.br/se-palmeiras/kader/verein/1023/plus/0/galerie/0?saison_id={temporada}', 'Palmeiras')
bragantino = (f'https://www.transfermarkt.com.br/rb-bragantino/kader/verein/8793/plus/0/galerie/0?saison_id={temporada}', 'Red Bull Bragantino')
santos = (f'https://www.transfermarkt.com.br/santos-fc/kader/verein/221/plus/0/galerie/0?saison_id={temporada}', 'Santos')
sao_paulo = (f'https://www.transfermarkt.com.br/sao-paulo-fc/kader/verein/585/plus/0/galerie/0?saison_id={temporada}', 'São Paulo')
gremio = (f'https://www.transfermarkt.com.br/gremio-fbpa/kader/verein/210/plus/0/galerie/0?saison_id={temporada}', 'Grêmio')
bahia = (f'https://www.transfermarkt.com.br/ec-bahia/kader/verein/10010/plus/0/galerie/0?saison_id={temporada}', 'Bahia')
sport = (f'https://www.transfermarkt.com.br/sport-recife/kader/verein/8718/plus/0/galerie/0?saison_id={temporada}', 'Sport Recife')
chapecoense = (f'https://www.transfermarkt.com.br/chapecoense/kader/verein/17776/plus/0/galerie/0?saison_id={temporada}', 'Chapecoense')

times = [america_mg, athletico, atletico_go, atletico_mg, ceara,
         corinthians, cuiaba, fluminense, flamengo, fortaleza,
         internacional, juventude, palmeiras, bragantino, santos,
         sao_paulo, gremio, bahia, sport, chapecoense]

In [1]:
import unicodedata

def remove_acentos(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [2]:
from pathlib import Path
import pandas as pd

def carrega_tabelas_arquivo():
    pasta = Path('C:/Users/Mathe/Desktop/trabalho-pl')
    
    transfermarkt = pd.read_csv(pasta / 'transfermarkt.csv')
    sofascore = pd.read_csv(pasta / 'sofascore.csv')
    
    return (transfermarkt, sofascore)

In [ ]:
from pathlib import Path

def salva_tabelas():
    pasta = Path('C:/Users/Mathe/Desktop/trabalho-pl')
    
    tabela_transfermarkt.to_csv(pasta / 'transfermarkt.csv')
    tabela_sofascore.to_csv(pasta / 'sofascore.csv')

In [3]:
def junta_tabelas(transfermarkt, sofascore):
    transfermarkt['Jogador'] = transfermarkt['Jogador'].str.lower()
    sofascore['Jogador'] = sofascore['Jogador'].str.lower()

    transfermarkt['Jogador'] = transfermarkt['Jogador'].map(remove_acentos)
    sofascore['Jogador'] = sofascore['Jogador'].map(remove_acentos)
    
    juncao = pd.merge(sofascore, transfermarkt, on=['Time', 'Jogador'], how='inner')
    
    juncao['Jogador'] = juncao['Jogador'].str.title()
    
    return juncao

In [4]:
juncao = junta_tabelas(*carrega_tabelas_arquivo())

In [5]:
juncao.loc[juncao['Posição'] == 'Meia Esquerda', 'Posição'] = 'Ala'
juncao.loc[juncao['Posição'] == 'Meia Direita', 'Posição'] = 'Ala'
juncao.loc[juncao['Posição'] == 'Seg. Atacante', 'Posição'] = 'Centroavante'

In [14]:
juncao['Rating Reserva'] = juncao['Rating'] * 0.5

In [7]:
juncao = juncao[['Time', 'Jogador', 'Posição', 'Preço', 'Rating', 'Colocação']]

In [8]:
juncao = juncao.sort_values(by='Colocação').reset_index(drop=True)

In [9]:
matriz_de_posicao = []
# GO LE ZA LD VO AL MC MO PE PD AT
posicoes = {'Goleiro': 0, 'Lateral Esq.': 1, 'Zagueiro': 2, 'Lateral Dir.': 3, 'Volante': 4,
           'Ala': 5, 'Meia Central': 6, 'Meia Ofensivo': 7, 'Ponta Esquerda': 8, 'Ponta Direita': 9,
           'Centroavante': 10}
for indice, linha in juncao.iterrows():
    matriz_de_posicao.append([0] * len(posicoes))
    
    matriz_de_posicao[indice][posicoes[linha['Posição']]] = 1

In [10]:
formacoes = [
   # GO LE ZA LD VO AL MC MO PE PD AT
    ([1, 1, 2, 1, 0, 2, 2, 0, 0, 0, 2], '4-4-2'),
    ([1, 1, 2, 1, 0, 0, 3, 0, 1, 1, 1], '4-3-3'),
    ([1, 1, 2, 1, 1, 0, 0, 2, 1, 1, 1], '4-3-3 ofensivo'),
    ([1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1], '4-3-3 equilibrado'),
    ([1, 1, 2, 1, 1, 2, 0, 1, 0, 0, 2], '4-4-2 diamante'),
    ([1, 1, 2, 1, 0, 2, 2, 1, 0, 0, 1], '4-5-1'),
    ([1, 1, 2, 1, 0, 0, 3, 2, 0, 0, 1], '4-3-2-1'),
    ([1, 1, 3, 1, 0, 0, 3, 0, 0, 0, 2], '5-3-2'),
    ([1, 0, 3, 0, 0, 2, 2, 0, 1, 1, 1], '3-4-3'),
    ([1, 0, 3, 0, 2, 2, 0, 1, 0, 0, 2], '3-5-2'),
    ([1, 1, 3, 1, 2, 0, 0, 1, 0, 0, 2], '5-3-2'),
    ([1, 0, 3, 0, 2, 2, 0, 2, 0, 0, 1], '3-6-1'),
    ([1, 1, 3, 1, 2, 0, 0, 2, 0, 0, 1], '5-4-1'),
    ([1, 1, 2, 1, 0, 2, 3, 0, 0, 0, 1], '4-5-1'),
    ([1, 1, 2, 1, 2, 2, 0, 1, 0, 0, 1], '4-2-3-1'),
    ([1, 1, 2, 1, 2, 0, 0, 1, 1, 1, 1], '4-2-1-3'),
    ([1, 1, 3, 1, 0, 2, 2, 0, 0, 0, 1], '5-4-1'),
    ([1, 1, 3, 1, 0, 0, 2, 0, 1, 1, 1], '5-2-3'),
    ([1, 1, 2, 1, 2, 0, 0, 2, 0, 0, 2], '4-2-2-2'),
]

for formacao, nome in formacoes:
    if sum(formacao) != 11:
        print('ERRO')

def melhor_time(investimento):
    ratings = []
    indice_max = 0
    rating_max = 0
    for indice, (formacao, nome) in enumerate(formacoes):
        rating, titulares, reservas = rating_max_formacao(formacao, investimento, matriz_de_posicao)
        
        preco_titular = sum(juncao['Preço'].iloc[titulares])
        preco_banco   = sum(juncao['Preço'].iloc[reservas])
        
        rating_titular = sum(juncao['Rating'].iloc[titulares])
        rating_banco   = sum(juncao['Rating'].iloc[reservas])
        
        ratings.append((rating_titular+rating_banco, titulares, reservas, nome, preco_titular+preco_banco))
        
        if rating > rating_max:
            rating_max = rating
            indice_max = indice
        
    return ratings[indice_max]

In [11]:
from numpy import zeros
def cria_tabela_pos(matriz):
    tabela_inicial = pd.DataFrame(matriz)
    
    metade1 = pd.concat([tabela_inicial, pd.DataFrame(zeros(tabela_inicial.shape))], axis='columns')
    metade2 = pd.concat([pd.DataFrame(zeros(tabela_inicial.shape)), tabela_inicial], axis='columns')
    
    resultado = pd.concat([metade1, metade2])
    resultado.columns = range(resultado.shape[1])
    
    return resultado.reset_index(drop=True)

In [12]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
inf = solver.infinity()

def cria_modelo(formacao, investimento, matriz_pos):
    dados = {}
    dados['coeficientes_restrição'] = [
        list(juncao['Preço']) + list(juncao['Preço']),
    ]
    # restringe pela posição do jogador. Com coluna = 0, restringe o goleiro, por exemplo
    tabela_pos = cria_tabela_pos(matriz_pos)
    for coluna in tabela_pos:
        dados['coeficientes_restrição'].append(list(tabela_pos[coluna]))
    
    # evita que jogadores duplicados possam aparecer no time. Uma restrição por jogador
    for i in range(len(juncao)):
        jogadores_duplicados = [0] * len(juncao)
        jogadores_duplicados[i] = 1
        
        dados['coeficientes_restrição'].append(jogadores_duplicados+jogadores_duplicados)
        
    dados['limites superiores'] = [investimento] + formacao * 2 + ([1]*len(juncao))
    dados['limites inferiores'] =            [0] + formacao * 2 + ([0]*len(juncao))
    
    dados['coeficientes objetivo'] = list(juncao['Rating']) + list(juncao['Rating Reserva'])
    
    dados['num_vars'] = len(juncao) * 2
    dados['num_restrições'] = len(dados['limites superiores'])

    return dados

def rating_max_formacao(formacao, investimento, matriz_pos):
    dados = cria_modelo(formacao, investimento, matriz_pos)
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    s = {}
    for j in range(dados['num_vars']):
        s[j] = solver.BoolVar(f's{j}')

    for i in range(dados['num_restrições']):
        constraint = solver.RowConstraint(dados['limites inferiores'][i], dados['limites superiores'][i], '')
        
        for j in range(dados['num_vars']):
            constraint.SetCoefficient(s[j], dados['coeficientes_restrição'][i][j])

    objective = solver.Objective()
    for j in range(dados['num_vars']):
        objective.SetCoefficient(s[j], dados['coeficientes objetivo'][j])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        jogadores = [j%452 for j in range(dados['num_vars']) if s[j].solution_value() == 1]
        return (solver.Objective().Value(), jogadores[:11], jogadores[11:])
    else:
        print('status ' + str(status) + ': ' + {1: 'FEASIBLE', 2: 'INFEASIBLE', 3: 'UNBOUNDED'}[status])

In [16]:
melhor_time(inf)

(158.71,
 [0, 1, 2, 4, 6, 8, 10, 16, 19, 24, 26],
 [3, 9, 11, 12, 14, 17, 23, 27, 29, 30, 42],
 '4-3-3 ofensivo',
 98.5)

In [17]:
juncao.iloc[[0, 1, 2, 4, 6, 8, 10, 16, 19, 24, 26]]

,Time,Jogador,Posição,Preço,Rating,Colocação,Rating Reserva
0,Flamengo,Giorgian De Arrascaeta,Meia Ofensivo,18.0,7.49,1,3.745
1,Palmeiras,Gustavo Scarpa,Meia Ofensivo,3.5,7.34,2,3.670
2,Red Bull Bragantino,Artur,Ponta Direita,5.0,7.33,3,3.665
4,Ceará,Joao Ricardo,Goleiro,0.4,7.32,5,3.660
6,Red Bull Bragantino,Leo Ortiz,Zagueiro,2.0,7.26,7,3.630
8,Atlético Mineiro,Hulk,Centroavante,2.5,7.24,9,3.620
10,Flamengo,Filipe Luis,Lateral Esq.,1.5,7.23,11,3.615
16,Flamengo,Bruno Henrique,Ponta Esquerda,4.5,7.18,17,3.590
19,São Paulo,Robert Arboleda,Zagueiro,1.6,7.16,20,3.580
24,Palmeiras,Marcos Rocha,Lateral Dir.,1.5,7.14,25,3.570


In [18]:
juncao.iloc[[3, 9, 11, 12, 14, 17, 23, 27, 29, 30, 42]]

,Time,Jogador,Posição,Preço,Rating,Colocação,Rating Reserva
3,Athletico,Nikao,Meia Ofensivo,1.40,7.32,4,3.660
9,Palmeiras,Raphael Veiga,Meia Ofensivo,4.50,7.23,10,3.615
11,Flamengo,Gabriel Barbosa,Centroavante,26.00,7.21,12,3.605
12,Sport Recife,Mailson,Goleiro,0.50,7.20,13,3.600
14,Fluminense,Danilo Barcelos,Lateral Esq.,0.85,7.18,15,3.590
17,América Mineiro,Ademir,Ponta Direita,0.50,7.17,18,3.585
23,Corinthians,Fagner,Lateral Dir.,3.00,7.14,24,3.570
27,Corinthians,Roger Guedes,Ponta Esquerda,9.00,7.13,28,3.565
29,Palmeiras,Gustavo Gomez,Zagueiro,6.00,7.12,30,3.560
30,Athletico,Thiago Heleno,Zagueiro,1.60,7.11,31,3.555


In [19]:
melhor_time(20)

(157.18,
 [3, 4, 6, 7, 10, 17, 19, 24, 31, 42, 53],
 [13, 14, 18, 20, 36, 37, 44, 49, 60, 63, 92],
 '4-3-3 equilibrado',
 19.949999999999996)

In [20]:
juncao.iloc[[3, 4, 6, 7, 10, 17, 19, 24, 31, 42, 53]]

,Time,Jogador,Posição,Preço,Rating,Colocação,Rating Reserva
3,Athletico,Nikao,Meia Ofensivo,1.40,7.32,4,3.660
4,Ceará,Joao Ricardo,Goleiro,0.40,7.32,5,3.660
6,Red Bull Bragantino,Leo Ortiz,Zagueiro,2.00,7.26,7,3.630
7,Internacional,Edenilson,Meia Central,2.40,7.25,8,3.625
10,Flamengo,Filipe Luis,Lateral Esq.,1.50,7.23,11,3.615
17,América Mineiro,Ademir,Ponta Direita,0.50,7.17,18,3.585
19,São Paulo,Robert Arboleda,Zagueiro,1.60,7.16,20,3.580
24,Palmeiras,Marcos Rocha,Lateral Dir.,1.50,7.14,25,3.570
31,Red Bull Bragantino,Ytalo,Centroavante,0.90,7.10,32,3.550
42,Sport Recife,Ze Welison,Volante,0.65,7.08,43,3.540


In [21]:
juncao.iloc[[13, 14, 18, 20, 36, 37, 44, 49, 60, 63, 92]]

,Time,Jogador,Posição,Preço,Rating,Colocação,Rating Reserva
13,Athletico,David Terans,Meia Ofensivo,1.40,7.19,14,3.595
14,Fluminense,Danilo Barcelos,Lateral Esq.,0.85,7.18,15,3.590
18,Internacional,Daniel,Goleiro,0.05,7.17,19,3.585
20,América Mineiro,Juninho Valoura,Meia Central,0.50,7.16,21,3.580
36,Juventude,Matheus Peixoto,Centroavante,0.70,7.08,37,3.540
37,Cuiabá,Joao Lucas,Lateral Dir.,0.30,7.08,38,3.540
44,Ceará,Luiz Otavio,Zagueiro,0.60,7.08,45,3.540
49,Atlético Mineiro,Nathan Silva,Zagueiro,0.60,7.07,50,3.535
60,Palmeiras,Felipe Melo,Volante,0.60,7.05,61,3.525
63,Red Bull Bragantino,Tomas Cuello,Ponta Esquerda,0.80,7.04,64,3.520
